In [1]:
%matplotlib inline
# notebook
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
from collections import Counter
import numpy as np
import pandas as pd
import networkx as nx
from math import sin, cos, sqrt, atan2, radians
from utils import *
from data import *
from trophics import *
from collections import defaultdict
#make the graphs bigger
pylab.rcParams['figure.figsize'] = (32.0, 24.0)
pylab.rcParams['font.size'] = 24

In [38]:
eiggRawData = eiggRawData()
eiggRawData = eiggRawData["Scientific name"].str.lower()
eiggRawData = set(eiggRawData)
len(eiggRawData)

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


3177

In [92]:
from io import StringIO
    
def standardiseNames(name):
    name = name.replace('"', '')
    name = name.replace("'",'')
    name = name.replace("?",'')
    name = name.split("/")
    name = name[0]
    name = re.sub(r'\([^)]*\)', '', name)
    name = name.strip()
    name = name.lower()
    return name

def getFileID(fileId):
    return standardiseNames(fileId.decode("utf-8")).split(" ")[0]

def prnFileReader():
    found = set()
    filesToProcess = getTestFiles()
    aggregated = []
    for f in filesToProcess:
        with open("./prnTest/"+f, 'rb') as file:
            content = (file.readlines())
        
        fileId = getFileID(content[0])
        
        content = content[1:-1]
        content = list(map(lambda x: x.strip().decode("utf-8"),content))
        content = list(map(lambda x: " ".join(x.split(" ")[1:3]), content))
        content = list(map(lambda x: standardiseNames(x),content))
        
        matrixCorresponding = None
        try:
            with open("./RelevantDatasets/ECOWeB1.1/ECOWeB1.1/DATFILES/COMMUNITY/WEB"+fileId+".DAT", 'rb') as matrix:
                matrixCorresponding = matrix.readlines()
        except Exception as e:
            continue
                    
        matrixCorresponding = matrixCorresponding[:-1]
        matrixCorresponding = list(map(lambda x: x.decode("utf-8").strip().split(" "),matrixCorresponding))
        matrixCorresponding = [list(filter(lambda y: len(y)>0,x)) for x in matrixCorresponding]
        matrixCorresponding = [list(map(lambda y: int(y),x)) for x in matrixCorresponding]
        
        colHeader = matrixCorresponding[0][1:]
        rowHeader = list(map(lambda x: x[0],matrixCorresponding[1:]))
        
        colHeader = list(map(lambda x: content[x-1],colHeader))
        rowHeader = list(map(lambda x: content[x-1],rowHeader))
        
        matrixCorresponding = list(map(lambda x: x[1:],matrixCorresponding))
        matrixCorresponding = matrixCorresponding[1:]

        data = pd.DataFrame(matrixCorresponding, index=rowHeader,columns=colHeader)
        return data
        
    return found

def getTestFiles():
    return os.listdir('C:/Users/davie/Desktop/Masters/Dissertation/Code/DissertationCode/Eigg/prnTest')    

prnFileReader()

,prawns and,"benthos(micro, meio",zooplankton herbivores,fish herbivores,other carnivores,fish carnivores,man
basic food,1,1,1,1,0,0,0
detritus,1,1,1,1,0,0,0
prawns and,0,0,0,0,1,1,1
"benthos(micro, meio",0,0,0,0,1,0,0
zooplankton herbivores,0,0,0,0,1,1,0
fish herbivores,0,0,0,0,0,1,1
other carnivores,0,0,0,0,0,1,0
fish carnivores,0,0,0,0,0,0,1


In [26]:
rawEiggData = validatedEiggData()
rawEiggData.head()

C:\Users\davie\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2862: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Recorder,Latitude (WGS84),Longitude (WGS84),Start date year,End date year,Scientific name,Common name
0,unknown,56.902729,-6.15936,1905.0,NaN,meloe violaceus,Violet Oil-beetle
1,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,rhantus suturellus,NaN
2,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,gyrinus substriatus,NaN
3,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,enochrus fuscipennis,NaN
4,Frank Balfour-Browne,56.899200,-6.19759,1910.0,1910.0,boreonectes multilineatus,NaN


In [3]:
foodWeb = retrieveCollatedFoodWeb()
len(foodWeb)

C:\Users\davie\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: DtypeWarning: Columns (2,8,12,13,22,26,27,41,44) have mixed types. Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
C:\Users\davie\Desktop\Masters\Dissertation\Code\DissertationCode\Eigg\trophics.py:20: DtypeWarning: Columns (9,22) have mixed types. Specify dtype option on import or set low_memory=False.
  dataSetFunctions = [readFreshwaterData(),read2018GlobalDatabaseData(),readSorensenData(),readJanesData()]


4284

In [4]:
overall = set()

for item in foodWeb:
    item = item.split(" ")
    item = " ".join(item[:2])
    overall.add(item)

for fw in foodWeb:
    v = foodWeb[fw]
    for item in v:
        item = item.split(" ")
        item = " ".join(item[:2])
        overall.add(item)

print(len(overall))

5495


In [31]:
eiggSpecies = rawEiggData['Scientific name'].str.lower()
eiggSpecies = set(eiggSpecies)

iset = set()
animalsTotal = len(eiggSpecies)
totalFound = 0
for item in eiggSpecies:
    if item in overall:
        iset.add(item)
        totalFound += 1
    

print(totalFound / animalsTotal)

0.09148665819567979


In [32]:
len(iset)

288